### 專案名稱: 第四屆機器學習百日馬拉松
### 功能描述: 第29天作業
### 版權所有: Dunk  
### 程式撰寫: Dunk  
### 撰寫日期：2020/03/31
### 改版日期:  
### 改版備註:

# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

In [8]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### ANS:

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [9]:
# 請先確認您的 sklearn 版本是否相同, 如果版本不是 0.21.1 版, 有可能跑出與本範例不同的結果
import sklearn
sklearn.__version__

'0.22'

In [10]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
col_name_obj = 'Cabin'
col_name_dec = 'Fare'
df[col_name_obj] = df[col_name_obj].fillna('None')
mean_df = df.groupby([col_name_obj])[col_name_dec].mean().reset_index()
mode_df = df.groupby([col_name_obj])[col_name_dec].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby([col_name_obj])[col_name_dec].median().reset_index()
max_df = df.groupby([col_name_obj])[col_name_dec].max().reset_index()
min_df = df.groupby([col_name_obj])[col_name_dec].min().reset_index()
count_df = df.groupby([col_name_obj])[col_name_dec].size().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=[col_name_obj])
temp = pd.merge(temp, median_df, how='left', on=[col_name_obj])
temp = pd.merge(temp, max_df, how='left', on=[col_name_obj])
temp = pd.merge(temp, min_df, how='left', on=[col_name_obj])
temp = pd.merge(temp, count_df, how='left', on=[col_name_obj])
temp.columns = [col_name_obj, col_name_dec+'_mean', col_name_dec+'_mode', col_name_dec+'_median', col_name_dec+'_max'\
               , col_name_dec+'_min', col_name_dec+'_count']
temp

,Cabin,Fare_mean,Fare_mode,Fare_median,Fare_max,Fare_min,Fare_count
0,A10,40.125000,40.1250,40.12500,40.1250,40.1250,1
1,A14,52.000000,52.0000,52.00000,52.0000,52.0000,1
2,A16,39.600000,39.6000,39.60000,39.6000,39.6000,1
3,A19,26.000000,26.0000,26.00000,26.0000,26.0000,1
4,A20,56.929200,56.9292,56.92920,56.9292,56.9292,1
...,...,...,...,...,...,...,...
143,F38,7.750000,7.7500,7.75000,7.7500,7.7500,1
144,F4,39.000000,39.0000,39.00000,39.0000,39.0000,2
145,G6,13.581250,10.4625,13.58125,16.7000,10.4625,4
146,None,19.157325,8.0500,10.50000,512.3292,0.0000,687


In [11]:
col_name_obj = 'Cabin'
df = pd.merge(df, temp, how='left', on=[col_name_obj])
df = df.drop([col_name_obj] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Fare_mean,Fare_mode,Fare_median,Fare_max,Fare_min,Fare_count
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,19.157325,8.0500,10.5000,512.3292,0.0000,687
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,71.283300,71.2833,71.2833,71.2833,71.2833,1
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,19.157325,8.0500,10.5000,512.3292,0.0000,687
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,53.100000,53.1000,53.1000,53.1000,53.1000,2
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,19.157325,8.0500,10.5000,512.3292,0.0000,687


In [12]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

11 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_mean', 'Fare_mode', 'Fare_median', 'Fare_max', 'Fare_min', 'Fare_count']



,Pclass,Age,SibSp,Parch,Fare,Fare_mean,Fare_mode,Fare_median,Fare_max,Fare_min,Fare_count
0,3,22.0,1,0,7.2500,19.157325,8.0500,10.5000,512.3292,0.0000,687
1,1,38.0,1,0,71.2833,71.283300,71.2833,71.2833,71.2833,71.2833,1
2,3,26.0,0,0,7.9250,19.157325,8.0500,10.5000,512.3292,0.0000,687
3,1,35.0,1,0,53.1000,53.100000,53.1000,53.1000,53.1000,53.1000,2
4,3,35.0,0,0,8.0500,19.157325,8.0500,10.5000,512.3292,0.0000,687


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?
### 結合後並無改善，應該是取的欄位有問題
>Fix
> 結果可以發現 : 不論是例題的線性迴歸或者梯度提升樹, 以及作業的邏輯斯迴歸  
聚類編碼都在正確率上有穩定提升, 這就是我們所說的:均值編碼容易overfitting/聚類編碼不容易overfitting的效果  
不過助教這邊的數值型特徵與類別型特徵, 是有特別用特徵重要性挑選過的, 因此同學自行挑選的特徵可能未必提升  
至於特徵重要性如何使用, 請同學參考之後 Day31 內容

In [13]:
# 原始特徵 + 邏輯斯迴歸
# 沒有這6個新特徵的 dataframe 稱為 df_minus
df_minus = df.drop([col_name_dec+'_mean', col_name_dec+'_mode', col_name_dec+'_median', col_name_dec+'_max'\
               , col_name_dec+'_min', col_name_dec+'_count'] , axis=1)
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_minus

0.70041428661101


,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500
...,...,...,...,...,...
886,2,27.0,0,0,13.0000
887,1,19.0,0,0,30.0000
888,3,-1.0,1,2,23.4500
889,1,26.0,0,0,30.0000


In [14]:
# 新特徵 + 邏輯斯迴歸
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df

0.6948088632226478


,Pclass,Age,SibSp,Parch,Fare,Fare_mean,Fare_mode,Fare_median,Fare_max,Fare_min,Fare_count
0,3,22.0,1,0,7.2500,19.157325,8.0500,10.5000,512.3292,0.0000,687
1,1,38.0,1,0,71.2833,71.283300,71.2833,71.2833,71.2833,71.2833,1
2,3,26.0,0,0,7.9250,19.157325,8.0500,10.5000,512.3292,0.0000,687
3,1,35.0,1,0,53.1000,53.100000,53.1000,53.1000,53.1000,53.1000,2
4,3,35.0,0,0,8.0500,19.157325,8.0500,10.5000,512.3292,0.0000,687
...,...,...,...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,19.157325,8.0500,10.5000,512.3292,0.0000,687
887,1,19.0,0,0,30.0000,30.000000,30.0000,30.0000,30.0000,30.0000,1
888,3,-1.0,1,2,23.4500,19.157325,8.0500,10.5000,512.3292,0.0000,687
889,1,26.0,0,0,30.0000,30.000000,30.0000,30.0000,30.0000,30.0000,1
